# Implementation - Factors and operations

In [1]:
import numpy as np

## 1 Factor data structure

Here factors are represented as multidimensional ```numpy``` arrays. Each axis corresponds to a particular random variable. Names of the variables are stored in ```self.__variables``` in the appropriate order.

For example, if we have a factor ```psi```, the value of $(a_2, b_1, c_3)$ can be accessed as follows: ```psi.get_distribution()[1,0,2]```

In [2]:
class factor:
    def __init__(self, variables = None, distribution = None):
        if (distribution is None) and (variables is not None):
            self.__set_data(np.array(variables), None, None)
        elif (variables is None) or (len(variables) != len(distribution.shape)):
            raise Exception('Data is incorrect')
        else:
            self.__set_data(np.array(variables),
                            np.array(distribution),
                            np.array(distribution.shape))
    
    def __set_data(self, variables, distribution, shape):
        self.__variables    = variables
        self.__distribution = distribution
        self.__shape        = shape
    
    # ----------------------- Info --------------------------
    def is_none(self):
        return True if self.__distribution is None else False
        
    # ----------------------- Getters -----------------------
    def get_variables(self):
        return self.__variables
    
    def get_distribution(self):
        return self.__distribution
    
    def get_shape(self):
        return self.__shape

---

## 2 Factor operations

There are several important operations on factors:

* factor product
* factor marginalization
* factor reduction
* computing joint distribution

### 2.1 Factor product

Representative example:

![](./img/2%20Implementation%20-%20Factors%20and%20operations/2.1.png)

<center>Fig 2.1: Figure is from [1]</center>

Here we are going to use ```numpy``` [broadcasting](https://docs.scipy.org/doc/numpy-1.17.0/user/basics.broadcasting.html) rules to perform tensor multiplication efficiently.

In [3]:
def factor_product(x, y):
    if x.is_none() or y.is_none():
        raise Exception('One of the factors is None')
    
    xy, xy_in_x_ind, xy_in_y_ind = np.intersect1d(x.get_variables(), y.get_variables(), return_indices=True)
    
    if xy.size == 0:
        raise Exception('Factors do not have common variables')
    
    if not np.all(x.get_shape()[xy_in_x_ind] == y.get_shape()[xy_in_y_ind]):
        raise Exception('Common variables have different order')
    
    x_not_in_y = np.setdiff1d(x.get_variables(), y.get_variables(), assume_unique=True)
    y_not_in_x = np.setdiff1d(y.get_variables(), x.get_variables(), assume_unique=True)
    
    x_mask = np.isin(x.get_variables(), xy, invert=True)
    y_mask = np.isin(y.get_variables(), xy, invert=True)
    
    x_ind = np.array([-1]*len(x.get_variables()), dtype=int)
    y_ind = np.array([-1]*len(y.get_variables()), dtype=int)
    
    x_ind[x_mask] = np.arange(np.sum(x_mask))
    y_ind[y_mask] = np.arange(np.sum(y_mask)) + np.sum(np.invert(y_mask))
    
    x_ind[xy_in_x_ind] = np.arange(len(xy)) + np.sum(x_mask)
    y_ind[xy_in_y_ind] = np.arange(len(xy))
    
    x_distribution = np.moveaxis(x.get_distribution(), range(len(x_ind)), x_ind)
    y_distribution = np.moveaxis(y.get_distribution(), range(len(y_ind)), y_ind)
                
    res_distribution =   x_distribution[tuple([slice(None)]*len(x.get_variables())+[None]*len(y_not_in_x))] \
                       * y_distribution[tuple([None]*len(x_not_in_y)+[slice(None)])]
    
    return factor(list(x_not_in_y)+list(xy)+list(y_not_in_x), res_distribution)

#### Example 1

Let's reproduce the example above.

In [4]:
phi_1 = factor(['a', 'b'], np.array([[0.5, 0.8], [0.1, 0.0], [0.3, 0.9]]))
phi_2 = factor(['b', 'c'], np.array([[0.5, 0.7], [0.1, 0.2]]))

phi_3 = factor_product(phi_1, phi_2)

In [5]:
phi_3.get_variables()

array(['a', 'b', 'c'], dtype='<U1')

In [6]:
phi_3.get_distribution()

array([[[0.25, 0.35],
        [0.08, 0.16]],

       [[0.05, 0.07],
        [0.  , 0.  ]],

       [[0.15, 0.21],
        [0.09, 0.18]]])

#### Example 2

Conditional probability distribution (CPD) and joint distribution can be represented as factors. Suppose we have $P(a|b)$ and $P(b)$:

|        |  $$P(a_1| b)$$   |  $$P(a_2|b)$$   |  $$P(a_3|b)$$   |
|---     |---               |---              |---              |
| $b_1$  |     0.3          |      0.2        |      0.5        |
| $b_2$  |     0.8          |      0.1        |      0.1        |

<center>Fig 2.2: CPD $P(a|b)$</center>

|        |  $$P(b)$$   |  
|---     |---          |
| $b_1$  |     0.3     |
| $b_2$  |     0.7     |

<center>Fig 2.3: P(b)</center>

Let's compute $P(a,b) = P(a|b)P(b)$

|        |  $a_1$    |  $a_2$    |  $a_3$   |
|---     |---        |---        |---       |
| $b_1$  |   0.09    |    0.06   |    0.15  |
| $b_2$  |   0.56    |    0.07   |    0.7   |

<center>Fig 2.4: Joint distribution P(a, b)</center>

In [7]:
phi_4 = factor(['a', 'b'], np.array([[0.3, 0.8], [0.2, 0.1], [0.5, 0.1]]))
phi_5 = factor(['b'], np.array([0.3, 0.7]))

phi_6 = factor_product(phi_1, phi_2)
phi_6.get_variables()

array(['a', 'b', 'c'], dtype='<U1')

In [8]:
phi_6.get_distribution()

array([[[0.25, 0.35],
        [0.08, 0.16]],

       [[0.05, 0.07],
        [0.  , 0.  ]],

       [[0.15, 0.21],
        [0.09, 0.18]]])

### 2.2 Factor marginalization

Representative example:

![](./img/2%20Implementation%20-%20Factors%20and%20operations/2.2.png)

<center>Fig 2.5: Figure is from [1]</center>

In [9]:
def factor_marginalization(x, variables):
    variables = np.array(variables)
    
    if x.is_none():
        raise Exception('Factor is None')
    
    if not np.all(np.in1d(variables, x.get_variables())):
        raise Exception('Factor do not contain given variables')
    
    res_variables    = np.setdiff1d(x.get_variables(), variables, assume_unique=True)
    res_distribution = np.sum(x.get_distribution(),
                              tuple(np.where(np.isin(x.get_variables(), variables))[0]))
    
    return factor(res_variables, res_distribution)

#### Example

Let's reproduce the example above.

In [10]:
phi_4 = factor_marginalization(phi_3, ['b'])

In [11]:
phi_4.get_variables()

array(['a', 'c'], dtype='<U1')

### 2.3 Factor reduction

Representative example:

![](./img/2%20Implementation%20-%20Factors%20and%20operations/2.3.png)

<center>Fig 2.6: Figure is from [1]</center>

In [12]:
def factor_reduction(x, variable, value):
    if x.is_none() or (variable is None) or (value is None):
        raise Exception('Input is None')
    
    if not np.any(variable == x.get_variables()):
        raise Exception('Factor do not contain given variable')
    
    if value >= x.get_shape()[np.where(variable==x.get_variables())[0]]:
        raise Exception('Incorrect value of given variable')
    
    res_variables    = np.setdiff1d(x.get_variables(), variable, assume_unique=True)
    res_distribution = np.take(x.get_distribution(),
                               value,
                               int(np.where(variable==x.get_variables())[0]))
    
    return factor(res_variables, res_distribution)

#### Example

Let's reproduce the example above.

In [13]:
phi_7 = factor_reduction(phi_3, 'c', 0)
phi_7.get_variables()

array(['a', 'b'], dtype='<U1')

In [14]:
phi_7.get_distribution()

array([[0.25, 0.08],
       [0.05, 0.  ],
       [0.15, 0.09]])

### 2.4 Joint distribution

In [15]:
def joint_distribution(ar):
    for element in ar:
        if element.is_none():
            raise Exception('Factor is None')
    
    res = ar[0]
    for element in ar[1:]:
        res = factor_product(res, element)
    
    return res

*References:*

1. D Koller "Probabilistic Graphical Models" by Stanford at Coursera [[link](https://ru.coursera.org/lecture/probabilistic-graphical-models/factors-tEZ6S)]